#### COMBINE FROM 10K which has valuable data

#### Installations

In [1]:
# Install vaex package
! pip install vaex


^C


In [ ]:
! pip install pyproject.toml-based projects

#### Imports

In [6]:
import pandas as pd
import os
import sqlite3

In [ ]:
pd.__version__

'2.0.1'

#### Constants

In [7]:
ticker_Path = 'firstDraft/processData'
base_dir = 'firstDraft/data-preprocessing/cleaning/Data/NVDA/processedData'
columns_to_keep = ['tag', 'type',  'value']

In [ ]:
def process_data(base_dir, company_name='APPLE INC', columns=None):
    results = []
    
    # Loop over each folder inside the processedData directory
    for folder_name in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder_name)
        if os.path.isdir(folder_path):
            # Read the sub.tsv file to get the submission IDs for the specified company
            sub_file_path = os.path.join(folder_path, 'sub.tsv')
            if os.path.exists(sub_file_path):
                sub_df = pd.read_csv(sub_file_path, sep='\t', on_bad_lines='skip', low_memory=False)
                if 'name' in sub_df.columns:
                    company_sub_ids = list(zip(sub_df[sub_df['name'] == company_name]['adsh'], sub_df[sub_df['name'] == company_name]['form']))
                    adsh_to_aciks = {adsh: aciks for adsh, aciks in company_sub_ids}
                    # Read the txt.tsv file and filter based on the submission IDs
                    txt_file_path = os.path.join(folder_path, 'txt.tsv')
                    folder_name = os.path.basename(folder_path)
                    if os.path.exists(txt_file_path):
                        txt_df = pd.read_csv(txt_file_path, sep='\t', on_bad_lines='skip', low_memory=False)
                        filtered_txt_df = txt_df[txt_df['adsh'].isin([adsh for adsh, _ in company_sub_ids])]
                        filtered_txt_df['type'] = filtered_txt_df['adsh'].map(sub_df.set_index('adsh')['form'])
                        if columns:
                            filtered_txt_df = filtered_txt_df[columns]
                        filtered_txt_df['time'] = os.path.basename(folder_name).replace('_notes', '')
                        filtered_txt_df['name'] = company_name
                        # Collect the filtered DataFrame
                        results.append(filtered_txt_df)
                else:
                    print(f"'company' column not found in {sub_file_path}")
            else:
                print(f"File not found: {sub_file_path}")
    # Combine all filtered DataFrames into one
    combined_df = pd.concat(results, ignore_index=True)
    return combined_df

# Define the base directory
base_dir = 'firstDraft/data-preprocessing/cleaning/Data/NVDA/processedData'

# Call the function and get the combined DataFrame
combined_df = process_data(base_dir, "AUTOMATIC DATA PROCESSING INC", columns_to_keep)

# Perform operations on the combined DataFrame
result = combined_df.head()
result

C:\Users\vecto\AppData\Local\Temp\ipykernel_35396\3089187761.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_txt_df['type'] = filtered_txt_df['adsh'].map(sub_df.set_index('adsh')['form'])
C:\Users\vecto\AppData\Local\Temp\ipykernel_35396\3089187761.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_txt_df['type'] = filtered_txt_df['adsh'].map(sub_df.set_index('adsh')['form'])
C:\Users\vecto\AppData\Local\Temp\ipykernel_35396\3089187761.py:21: SettingWithCopyWarning: 
A value is tryi

,tag,type,value,time,name
0,FairValueMeasurementPolicyPolicyTextBlock,8-K,Fair Value Measurements The Company applies fa...,2013q2,APPLE INC
1,ScheduleOfAccruedLiabilitiesTableTextBlock,8-K,Accrued Expenses 2012 2011 Accrued warranty an...,2013q2,APPLE INC
2,FiniteLivedIntangibleAssetUsefulLife,8-K,P7Y,2013q2,APPLE INC
3,TradingSymbol,8-K,AAPL,2013q2,APPLE INC
4,ScheduleOfNonvestedRestrictedStockUnitsActivit...,10-Q,A summary of the Companys RSU activity and rel...,2013q2,APPLE INC


In [9]:
combined_df.shape

(7563, 5)

In [10]:
conn = sqlite3.connect('firstDraft/data-preprocessing/cleaning/Data/dataBase/processdDB.db')

In [11]:
combined_df.to_sql('processdDB', conn, if_exists='append', index=False)


7563

In [12]:
conn.close()